In [291]:
#required packages and variables

import os 
import sys
import pandas as pd
import string
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score
import numpy as np

#variables
raw_train = "df_train_0.csv"
raw_val = "df_val_0.csv"
test = "df_test_0.csv"
classifier_loc = "/Users/inkyunpark/vscjava/rdptools/classifier.jar"
training_files_loc = "training_files"
classified_loc = "output.txt"
confidence_score = 0.8
level = 'genus' #ranks = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']

In [298]:
#NECESSARY DEFINITIONS FOR RDP TRAINING, CLASSIFICATION AND SCORING

#codes to make raw sequence and raw taxonomy into ready4rdp input files.
#scripts are from https://github.com/GLBRC-TeamMicrobiome/python_scripts and revised to be used in this concatenated jupyter notebook file.

def lineage2taxTrain(raw_taxons):
    taxons_list = raw_taxons.strip().split('\n')
    header = taxons_list[0].split('\t')[1:]#headers = list of ranks
    hash = {}#taxon name-id map
    ranks = {}#column number-rank map
    lineages = []#list of unique lineages

    with open("RDP_files/ready4train_taxonomy.txt", "w") as f:
        hash = {"Root":0}#initiate root rank taxon id map
        for i in range(len(header)):
            name = header[i]
            ranks[i] = name
        root = ['0', 'Root', '-1', '0', 'rootrank']#root rank info
        f.write("*".join(root) +  '\n')
        ID = 0 #taxon id
        for line in taxons_list[1:]:
            cols = line.strip().split('\t')[1:]
            for i in range(len(cols)):#iterate each column
                name = []
                for node in cols[:i + 1]:
                    node = node.strip()
                    if not node in ('-', ''):
                        name.append(node)
                pName = ";".join(name[:-1])
                if not name in lineages:
                    lineages.append(name)
                depth = len(name)
                name = ';'.join(name)
                if name in hash.keys():#already seen this lineage
                    continue
                try:
                    rank = ranks[i]
                except KeyError:
                    print (cols)
                    sys.exit()
                if i == 0:
                    pName = 'Root'
                pID = hash[pName]#parent taxid
                ID += 1
                hash[name] = ID #add name-id to the map
                out = ['%s'%ID, name.split(';')[-1], '%s'%pID, '%s'%depth, rank]
                f.write("*".join(out) + '\n')
    f.close()

def addFullLineage(raw_taxons, raw_seqs):
    hash = {} #lineage map
    taxonomy_list = raw_taxons.strip().split('\n')
    for line in taxonomy_list[1:]:
        line = line.strip()
        cols = line.strip().split('\t')
        lineage = ['Root']
        for node in cols[1:]:
            node = node.strip()
            if not (node == '-' or node == ''):
                lineage.append(node)
        ID = cols[0]
        lineage = ';'.join(lineage).strip()
        hash[ID] = lineage
    sequence_list = raw_seqs.strip().split('\n')
    with open("RDP_files/ready4train_seqs.fasta", "w") as f:
        for line in sequence_list:
            line = line.strip()
            if line == '':
                continue
            if line[0] == '>':
                ID = line.strip().split()[0].replace('>', '')
                lineage = hash[ID]
                f.write('>' + ID + '\t' + lineage + '\n')
            else:
                f.write(line.strip() + '\n')
    f.close()

def RDPoutput2score(pred_file, true_file, level, cf):
    taxon_list = ['unclassified']
    ranks = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
    level = ranks.index(level)

    pred_file_loc = 'RDP_files/' + pred_file
    pred = pd.read_csv(pred_file_loc, sep="\t", header=None)
    pred.drop(pred.columns[1:level+5+2*level],  axis = 'columns', inplace=True)
    pred.drop(pred.columns[4:], axis = 'columns', inplace=True)
    
    pred_dict = {}
    for index, row in pred.iterrows():
        row = row.tolist()
        if row[1] not in taxon_list:
            taxon_list += [row[1]]
        if float(row[3]) >= cf:
            pred_dict[row[0]] = row[1]

    true = pd.read_csv(true_file, sep="\t", header=None)
    true_dict = {}
    for index, row in true.iterrows():
        true_dict[row[0]] = row[level+1]
        if row[level+1] not in taxon_list:
            taxon_list += [row[level+1]]


    y_pred, y_true = [], []
    for i in pred_dict.keys():
        y_pred.append(taxon_list.index(pred_dict[i]))
        y_true.append(taxon_list.index(true_dict[i]))

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted') # works well if both y_true_decode and y_pred are list.
    mcc = matthews_corrcoef(y_true, y_pred)

    print("accuracy is {}\nf1 score is {}\nMCC score is {}".format(acc, f1, mcc))

In [293]:
#main_RDP.py

os.system("mkdir RDP_files")

#merge train and validation database into one.
train = pd.concat(map(pd.read_csv, [raw_train, raw_val]))
test = pd.read_csv(test)

#change train and test dataframe into taxonomy and sequence strings
raw_seqs = ''
raw_taxons = 'SeqId	Kingdom	Phylum	Class	Order	Family	Genus	Species' + '\n'
for index, row in train.iterrows():
    taxons = row.tolist()
    raw_seqs += '>' + taxons[0] + '\n' + taxons[-1] + '\n'
    raw_taxons += '\t'.join(taxons[:-1]) + '\n'

#change raw taxonomy and sequence files to ready4rdp trainable files
lineage2taxTrain(raw_taxons)
addFullLineage(raw_taxons, raw_seqs)

#change test dataframe to test sequence files to be classified by RDP
with open("RDP_files/test_sequences.fasta", "w") as seq_f, open("RDP_files/test_taxonomy.txt", "w") as tax_f:
    for index,row in test.iterrows():
        taxons = row.tolist()
        seq_f.write('>' + taxons[0] + '\n' + taxons[-1] + '\n')
        tax_f.write('\t'.join(taxons[:-1]) + '\n')
    seq_f.close()
    tax_f.close()

#train RDP models and add necessary file.
os.system("java -Xmx10g -jar {} train -o RDP_files/{} -s RDP_files/ready4train_seqs.fasta -t RDP_files/ready4train_taxonomy.txt".format(classifier_loc, training_files_loc))
with open("RDP_files/{}/rRNAClassifier.properties".format(training_files_loc), "w") as f:
    f.write("bergeyTree=bergeyTrainingTree.xml" + '\n' + "probabilityList=genus_wordConditionalProbList.txt" + '\n' + "probabilityIndex=wordConditionalProbIndexArr.txt" + '\n' + "wordPrior=logWordPrior.txt" + '\n' + "classifierVersion=RDP Naive Bayesian rRNA Classifier Version 2.5, May 2012 ")
    f.close()

#classify test sequences
os.system("java -Xmx10g -jar {} classify -t RDP_files/{}/rRNAClassifier.properties  -o RDP_files/{} RDP_files/test_sequences.fasta".format(classifier_loc, training_files_loc, classified_loc))

#scoring
RDPoutput2score(classified_loc, "RDP_files/test_taxonomy.txt", level, confidence_score)

FileNotFoundError: [Errno 2] No such file or directory: 'output.txt'